In [4]:
from external_lib.visualize import visualize_trafic
import pandas as pd
from lib.kalman_filter import apply_kalmanfilter,generate_kalmanfilter

from multiplyList import find_closest_using_single_point

from lib.noglobal import noglobal

import warnings
warnings.simplefilter('ignore')
from shapely.geometry import Point
import osmnx as ox
import momepy
import geopandas as gpd

from tqdm.notebook import tqdm

import numpy as np

from external_lib.evaluation_function import calc_haversine

In [2]:
@noglobal()
def aasdf(x,grid,i):
    return np.float64(grid[x][i]);



@noglobal()
def generate_grids(arg_df,target_collection):
    df  = arg_df.copy();
    df = arg_df[arg_df["collectionName"]==target_collection]
        
    df["geometry"] = [ Point(p) for p in df[["lngDeg","latDeg"]].to_numpy()]
    ground_df = gpd.GeoDataFrame(df, geometry=df["geometry"])
    
    offset = 0.1**4
    bbox = ground_df.bounds + [-offset, -offset, offset, offset]
    
    east = bbox["minx"].min()
    west = bbox["maxx"].max()
    south = bbox["miny"].min()
    north = bbox["maxy"].max()
    G = ox.graph.graph_from_bbox(north, south, east, west, network_type='drive')
    
    original_nodes, original_edges = momepy.nx_to_gdf(G)
    
    edges = original_edges.dropna(subset=["geometry"]).reset_index(drop=True)
    
    hits = bbox.apply(lambda row: list(edges.sindex.intersection(row)), axis=1)
    
    tmp = pd.DataFrame({
    "pt_idx":np.repeat(hits.index,hits.apply(len)),
    "line_i":np.concatenate(hits.values)        
    })
    
    tmp = tmp.join(edges.reset_index(drop=True),on = "line_i")

    tmp = tmp.join(ground_df.geometry.rename("point"),on = "pt_idx")

    tmp = gpd.GeoDataFrame(tmp, geometry="geometry", crs=ground_df.crs)
    
    tmp["snap_dist"] = tmp.geometry.distance(gpd.GeoSeries(tmp["point"]))


    tolerance = 0.0005
    tmp = tmp.loc[tmp["snap_dist"] <= tolerance]
    tmp = tmp.sort_values(by = ["snap_dist"]);

    closest = tmp.groupby("pt_idx").first()

    cloest = gpd.GeoDataFrame(closest, geometry = "geometry")


    closest = gpd.GeoDataFrame(cloest, geometry = "geometry");
    closest =closest.drop_duplicates("line_i").reset_index(drop=True)
    
    
    line_points_list = []
    split = 50  # param: number of split in each LineString
    for dist in range(0, split, 1):
        dist = dist/split
        line_points = closest["geometry"].interpolate(dist, normalized=True)
        line_points_list.append(line_points)
        line_points = pd.concat(line_points_list).reset_index(drop=True)
        line_points = line_points.reset_index().rename(columns={0:"geometry"})
        line_points["lngDeg"] = line_points["geometry"].x
        line_points["latDeg"] = line_points["geometry"].y
        
    return line_points

In [9]:
df_train = load_pickle_data("/work/data/input/selfmade_dataset/baseline_with_derived_data_v5/train.pkl");

pd_list = [];
bef_collectionName="";

for key,each_df in tqdm(df_train.groupby(["collectionName","phoneName"])):
    tmp_df = each_df.copy();
    num =  tmp_df[["latDeg","lngDeg"]].to_numpy().tolist()
    
    collectionName = key[0]
    
    if (not (bef_collectionName == collectionName)):
            line_points = generate_grids(df_train,collectionName);
            grids_list = line_points[["latDeg","lngDeg"]].to_numpy().tolist()

    
    tmp_df["min_idx"] = find_closest_using_single_point(num,grids_list);
    
    tmp_df["latDeg"] = tmp_df["min_idx"].apply(lambda x: aasdf(x,grids_list,0))
    tmp_df["lngDeg"] = tmp_df["min_idx"].apply(lambda x: aasdf(x,grids_list,1))
    
    pd_list.append(tmp_df)
                
    
result_df = pd.concat(pd_list).sort_index();

  0%|          | 0/73 [00:00<?, ?it/s]

In [19]:
from lib.io import load_pickle_data


result_df["dist"] = calc_haversine(result_df["latDeg"],result_df["lngDeg"],gt["latDeg_gt"],gt["lngDeg_gt"]);
df_train["dist"] = calc_haversine(df_train["latDeg"],df_train["lngDeg"],gt["latDeg_gt"],gt["lngDeg_gt"]);
print("before",df_train["dist"].mean())
print("after",result_df["dist"].mean())

for key,each_df in tqdm(result_df.groupby(["collectionName","phoneName"])):
    df_train_per_path = df_train[df_train["phone"] == key[0] + "_"+key[1]];
    
    print("-----------------------------------")
    print(key[0] + "_"+key[1]);    
    print("before",df_train_per_path["dist"].mean())
    print("after",each_df["dist"].mean())

    
    
    
    

before 3.8468553394627825
after 13.990995328053279


  0%|          | 0/73 [00:00<?, ?it/s]

-----------------------------------
2020-05-14-US-MTV-1_Pixel4
before 1.4564586662027643
after 9.959998664886395
-----------------------------------
2020-05-14-US-MTV-1_Pixel4XLModded
before 2.2339342621224874
after 10.144809467276728
-----------------------------------
2020-05-14-US-MTV-2_Pixel4
before 1.4954376196884398
after 9.200284894158745
-----------------------------------
2020-05-14-US-MTV-2_Pixel4XLModded
before 5.077656171933671
after 8.983170446503147
-----------------------------------
2020-05-21-US-MTV-1_Pixel4
before 2.182171381469512
after 15.236756232346409
-----------------------------------
2020-05-21-US-MTV-2_Pixel4
before 1.2982310644205972
after 20.42967967661566
-----------------------------------
2020-05-21-US-MTV-2_Pixel4XL
before 1.43696904330486
after 8.16866551793255
-----------------------------------
2020-05-29-US-MTV-1_Pixel4
before 3.8450361884593347
after 11.74691681835394
-----------------------------------
2020-05-29-US-MTV-1_Pixel4XL
before 2.3069716